# Interactive Hierarchical Monitoring Visualizations

This notebook provides interactive visualizations of the hierarchical monitoring system's decision boundaries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, VBox, HBox
import IPython.display as display

def sigmoid(x):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

In [ ]:
def plot_decision_boundaries(u=2.0, l=-0.5, beta_u=3.0, beta_l=3.0,
                            alpha_1=0.4, alpha_2=0.65, beta_3=3.0, a=0.1,
                            c_monitor=1.0, c_audit=20.0, budget=2.0):
    """
    Plot decision boundaries with monitor score on x-axis and probe score on y-axis.
    Includes beta_3 and a for p3 = sigmoid(beta_3 * (m3 - a))
    """
    # Create grid (note: swapped axes)
    monitor_range = np.linspace(-4, 4, 100)  # x-axis
    probe_range = np.linspace(-4, 4, 100)    # y-axis
    M2, M1 = np.meshgrid(monitor_range, probe_range)

    # Compute probabilities
    p1 = sigmoid(beta_u * (M1 - u))
    p2 = sigmoid(beta_l * (M1 - l))
    m3 = alpha_1 * M1 + alpha_2 * M2
    # Updated p3 with beta_3 and a
    p3 = sigmoid(beta_3 * (m3 - a))

    p_gate = (1 - p1) * p2
    p_audit = p1 + p_gate * p3

    # Compute cost
    cost = c_monitor * p_gate + c_audit * p_audit
    cost_ratio = cost / budget

    # Create figure with 3 subplots
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Plot 1: P(audit)
    ax1 = axes[0]
    im1 = ax1.contourf(M2, M1, p_audit, levels=20, cmap='Blues')
    ax1.axhline(y=u, color='red', linestyle='--', linewidth=2, label=f'u={u:.1f}')
    ax1.axhline(y=l, color='green', linestyle='--', linewidth=2, label=f'l={l:.1f}')

    # Add ensemble boundary (m3 = a, where p3 = 0.5)
    if alpha_1 != 0:
        monitor_line = np.linspace(-4, 4, 100)
        # alpha_1 * m1 + alpha_2 * m2 = a
        # m1 = (a - alpha_2 * m2) / alpha_1
        probe_line = (a - alpha_2 * monitor_line) / alpha_1
        mask = (probe_line >= -4) & (probe_line <= 4)
        ax1.plot(monitor_line[mask], probe_line[mask], 'orange', linestyle=':',
                linewidth=2, label=f'm₃={a:.1f}')

    ax1.contour(M2, M1, p_audit, levels=[0.5, 0.8, 0.95], colors='white', linewidths=1, alpha=0.5)
    ax1.set_xlabel('Monitor Score (m₂)')
    ax1.set_ylabel('Probe Score (m₁)')
    ax1.set_title('P(audit) - Total Audit Probability')
    ax1.legend(loc='upper right', fontsize=8)
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim([-4, 4])
    ax1.set_ylim([-4, 4])
    plt.colorbar(im1, ax=ax1, label='P(audit)')

    # Plot 2: P(monitor call)
    ax2 = axes[1]
    im2 = ax2.contourf(M2, M1, p_gate, levels=20, cmap='Greens')
    ax2.axhline(y=u, color='red', linestyle='--', linewidth=2)
    ax2.axhline(y=l, color='green', linestyle='--', linewidth=2)
    ax2.contour(M2, M1, p_gate, levels=[0.25, 0.5, 0.75], colors='white', linewidths=1, alpha=0.5)
    ax2.set_xlabel('Monitor Score (m₂)')
    ax2.set_ylabel('Probe Score (m₁)')
    ax2.set_title('P(gate) - Monitor Call Probability')
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim([-4, 4])
    ax2.set_ylim([-4, 4])
    plt.colorbar(im2, ax=ax2, label='P(gate)')

    # Plot 3: Expected Cost/Budget (with Reds colormap like original)
    ax3 = axes[2]
    max_cost = max(c_monitor + c_audit, 25)  # Dynamic max based on costs
    im3 = ax3.contourf(M2, M1, cost, levels=20, cmap='Reds', vmin=0, vmax=max_cost)
    ax3.axhline(y=u, color='red', linestyle='--', linewidth=2)
    ax3.axhline(y=l, color='green', linestyle='--', linewidth=2)

    # Add ensemble boundary
    if alpha_1 != 0:
        monitor_line = np.linspace(-4, 4, 100)
        probe_line = (a - alpha_2 * monitor_line) / alpha_1
        mask = (probe_line >= -4) & (probe_line <= 4)
        ax3.plot(monitor_line[mask], probe_line[mask], 'orange', linestyle=':',
                linewidth=2, label=f'm₃={a:.1f}')

    # Contour at budget value
    ax3.contour(M2, M1, cost, levels=[budget], colors='blue', linewidths=3, linestyles='--')
    # Additional cost contours
    contour_levels = [budget/2, budget*2, budget*3]
    ax3.contour(M2, M1, cost, levels=contour_levels, colors='black', linewidths=0.5, alpha=0.5)

    ax3.set_xlabel('Monitor Score (m₂)')
    ax3.set_ylabel('Probe Score (m₁)')
    ax3.set_title(f'Expected Cost (Budget=${budget:.1f})')
    ax3.legend(loc='upper right', fontsize=8)
    ax3.grid(True, alpha=0.3)
    ax3.set_xlim([-4, 4])
    ax3.set_ylim([-4, 4])
    plt.colorbar(im3, ax=ax3, label='Cost ($)')

    plt.suptitle('Hierarchical Monitoring System - Decision Boundaries', fontsize=16, y=1.05)
    plt.tight_layout()
    plt.show()

    # Print current parameter values
    print(f"Gate parameters: u={u:.2f}, l={l:.2f}, β_u={beta_u:.1f}, β_l={beta_l:.1f}")
    print(f"Ensemble parameters: α₁={alpha_1:.2f}, α₂={alpha_2:.2f}, β₃={beta_3:.1f}, a={a:.2f}")
    print(f"Cost structure: Monitor=${c_monitor:.1f}, Audit=${c_audit:.1f}, Budget=${budget:.1f}")

In [ ]:
# Create interactive widget with sliders
def create_interactive_plot():
    """Create interactive plot with sliders for Colab."""

    interact(plot_decision_boundaries,
             u=FloatSlider(value=2.0, min=-2, max=3, step=0.1, description='u (upper):',
                          continuous_update=False, style={'description_width': 'initial'}),
             l=FloatSlider(value=-0.5, min=-3, max=2, step=0.1, description='l (lower):',
                          continuous_update=False, style={'description_width': 'initial'}),
             beta_u=FloatSlider(value=3.0, min=0.5, max=10, step=0.5, description='β_u:',
                               continuous_update=False, style={'description_width': 'initial'}),
             beta_l=FloatSlider(value=3.0, min=0.5, max=10, step=0.5, description='β_l:',
                               continuous_update=False, style={'description_width': 'initial'}),
             alpha_1=FloatSlider(value=0.4, min=0, max=1, step=0.05, description='α₁ (probe):',
                                continuous_update=False, style={'description_width': 'initial'}),
             alpha_2=FloatSlider(value=0.65, min=0, max=1, step=0.05, description='α₂ (monitor):',
                                continuous_update=False, style={'description_width': 'initial'}),
             beta_3=FloatSlider(value=3.0, min=0.5, max=10, step=0.5, description='β₃ (ensemble):',
                               continuous_update=False, style={'description_width': 'initial'}),
             a=FloatSlider(value=0.1, min=-2, max=2, step=0.1, description='a (ensemble threshold):',
                          continuous_update=False, style={'description_width': 'initial'}),
             c_monitor=FloatSlider(value=1.0, min=0.1, max=5, step=0.1, description='Monitor Cost ($):',
                                   continuous_update=False, style={'description_width': 'initial'}),
             c_audit=FloatSlider(value=20.0, min=1, max=50, step=1, description='Audit Cost ($):',
                                continuous_update=False, style={'description_width': 'initial'}),
             budget=FloatSlider(value=2.0, min=0.5, max=10, step=0.5, description='Budget ($):',
                               continuous_update=False, style={'description_width': 'initial'}))

# Run the interactive plot
create_interactive_plot()

## Run Curriculum Learning Experiment

The following cell runs the curriculum learning experiment with the proper recall-cost loss function.

In [ ]:
# Import the curriculum learning functions
exec(open('curriculum_experiment.py').read())

# Run the experiment
model, history = run_curriculum_experiment()